语义分割-FCN-8s

tinymind 模型训练网址https://www.tinymind.com/yinyuyang/fcn-8s/code/train.py

以下是fcn-8s代码的部分

In [ ]:
upsample_factor = 8
number_of_classes = 21

log_folder = os.path.join(FLAGS.output_dir, 'train')

vgg_checkpoint_path = FLAGS.checkpoint_path

# Creates a variable to hold the global_step.
global_step = tf.Variable(0, trainable=False, name='global_step', dtype=tf.int64)


# Define the model that we want to use -- specify to use only two classes at the last layer
with slim.arg_scope(vgg.vgg_arg_scope()):
    logits, end_points = vgg.vgg_16(image_tensor,
                                    num_classes=number_of_classes,
                                    is_training=is_training_placeholder,
                                    spatial_squeeze=False,
                                    fc_conv_padding='SAME')

downsampled_logits_shape = tf.shape(logits)

img_shape = tf.shape(image_tensor)

# Calculate the ouput size of the upsampled tensor
# The shape should be batch_size X width X height X num_classes
upsampled_logits_shape = tf.stack([
                                  downsampled_logits_shape[0],
                                  img_shape[1],
                                  img_shape[2],
                                  downsampled_logits_shape[3]
                                  ])

#vgg_16中 pool3 输出是28*28*256，pool4 输出是14*14*512
#将pool3 和 pool4 的feature map 通过卷积进行分类，分别变成，28*28*21、14*14*21
pool4_feature = end_points['vgg_16/pool4']
pool3_feature = end_points['vgg_16/pool3']
with tf.variable_scope('vgg_16/fc8'):
    aux_logits_16s = slim.conv2d(pool4_feature, number_of_classes, [1, 1],
                                 activation_fn=None,
                                 weights_initializer=tf.zeros_initializer,
                                 scope='conv_pool4')
#with tf.variable_scope('vgg_16/fc'):
    aux_logits_8s = slim.conv2d(pool3_feature, number_of_classes, [1, 1],
                                 activation_fn=None,
                                 weights_initializer=tf.zeros_initializer,
                                 scope='conv_pool3')
    
# Perform the upsampling
#以下相当于先进行了16s的操作，包括一次2倍的上采样，这一部分输出的是28*28*21
upsample_filter_np_x2 = bilinear_upsample_weights(2,  # upsample_factor,
                                                  number_of_classes)

upsample_filter_tensor_x2 = tf.Variable(upsample_filter_np_x2, name='vgg_16/fc8/t_conv_x2')

upsampled_logits_16 = tf.nn.conv2d_transpose(logits, upsample_filter_tensor_x2,
                                          output_shape=tf.shape(aux_logits_16s),
                                          strides=[1, 2, 2, 1],
                                          padding='SAME')


upsampled_logits_16 = upsampled_logits_16 + aux_logits_16s


#在16s的基础上，再进行2倍上采样与pool3 的结果加和,这一部分的输出是14*14*21
upsample_filter_tensor_x2_x2 = tf.Variable(upsample_filter_np_x2, name='vgg_16/fc8/t_conv_x2_x2')
upsampled_logits_8 = tf.nn.conv2d_transpose(upsampled_logits_16, upsample_filter_tensor_x2_x2,
                                          output_shape=tf.shape(aux_logits_8s),
                                          strides=[1, 2, 2, 1],
                                          padding='SAME')

upsampled_logits_8 = upsampled_logits_8 + aux_logits_8s

#最后进行8倍上采样，将输出变成与原图大小相同的分类结果
upsample_filter_np_x8 = bilinear_upsample_weights(upsample_factor,
                                                   number_of_classes)

upsample_filter_tensor_x8 = tf.Variable(upsample_filter_np_x8, name='vgg_16/fc8/t_conv_x8')

    
upsampled_logits = tf.nn.conv2d_transpose(upsampled_logits_8, upsample_filter_tensor_x8,
                                          output_shape=upsampled_logits_shape,
                                          strides=[1, upsample_factor, upsample_factor, 1],
                                          padding='SAME')

lbl_onehot = tf.one_hot(annotation_tensor, number_of_classes)
cross_entropies = tf.nn.softmax_cross_entropy_with_logits(logits=upsampled_logits,
                                                          labels=lbl_onehot)

cross_entropy_loss = tf.reduce_mean(tf.reduce_sum(cross_entropies, axis=-1))


数据集，voc2012提供的有21张左右的图片大小是不符合vgg的默认图片（224*224）大小的，不是标准的横拍图片或图片本身太小的，在制作数据集的过程中直接删除了

* 对fcn-8s 和 fcn-16s 都分别用batchsize 8、16，steps3000 跑了一下，

* batchsize=8时 两种速度基本持平，loss最后的收敛都不好在100左右，不过fcn-8s的loss下降曲线要更稳一些，fcn-16s loss曲线有较大的振荡，可能是因为fcn-8s利用了更多的feature

* batchsize=16，两种收敛都快了很多，loss也更低

* 在看结果的时候，检测结果对物体在图片中心而且比例比较大的识别较好，而对小物体（鸟类识别效果很差），有重叠的物体识别就不是很好

* crf在训练已经有大概轮廓的时候能明显看到它的作用，将物体的轮廓描绘的更加细致和准确，尤其是针对背景很干净的图片

* fcn的思路是在原来分类网络的基础上，将对整张图片的分类扩展成像素级的分类，特征提取后再扩大feature map到原图大小，

* fcn主要用了转置卷积、CRF和跳阶结构，CRF不参与优化，直观上想，应该最后上采样的倍数越小越好，因为上采样是比较机械的扩大featue map没有利用到更多的信息，但是在这次尝试中没有看到二者有特别显著区别